# Simulation of the ferromagnet CrI$_3$, SQW4-component

The files necessary to perform the simulation are generated using the SpinW software, and the exchange parameters and model are taken from literature. 

In [1]:
#More complex example of pyMACS usage

import numpy as np
import pyMACS
from pyMACS.virtualMACS import VirtualMACS
import mcstasscript as ms

macs = VirtualMACS('cri3_experiment',cifName='CrI3.cif',useOld=False)
macs.sample.formula_weight=432.7
numthreads=8


This is a more complex example of the usage of the pyMACS interface, which 
simulates a real experiment from start to finish. The system of choice is 
CrI3, which has a well defined spin hamiltonian. The system was simulated using
SpinW, with the spin interactions defined by the following work 

https://journals.aps.org/prx/abstract/10.1103/PhysRevX.8.041028

The sample is assumed to be larger than is realistic, to give appreciable intensity
from spin waves. 

In [2]:
#Going to make our sample artifically big to increase count rate. 
macs.sample.sample_widx=0.01
macs.sample.sample_widy=0.01
macs.sample.sample_widz=0.01

macs.monochromator.Ei = 5.0
macs.monochromator.Ef = 5.0
macs.sample.cif2lau()

#Sample is aligned in the HK0 plane
macs.sample.orient_u=[1,0,0]
macs.sample.orient_v=[-1,2,0]
macs.sample.project_sample_realspace()
print('Real Space projection of lattice vectors [ax,ay,az; bx,by,bz;cx,cy,cz]')
print(macs.sample.labframe_mat)
print('')
print('Structure factors:')
print('|F(110)|^2 = '+str(round(macs.sample.fetch_F_HKL(1,1,0)[3],4))+' barn')
print('|F(100)|^2 = '+str(round(macs.sample.fetch_F_HKL(1,0,0)[3],4))+' barn')
print('|F(1-10)|^2 = '+str(round(macs.sample.fetch_F_HKL(1,-1,0)[3],4))+' barn')
print('|F(001)|^2 = '+str(round(macs.sample.fetch_F_HKL(0,0,1)[3],4))+' barn')

print('sigma_abs ='+str(macs.sample.rho_abs)+' barn/unit cell')
print('sigma_inc ='+str(macs.sample.sigma_inc)+' barn/unit cell')

 
Conversion of CIF to crystallographical LAU file successful. 
Real Space projection of lattice vectors [ax,ay,az; bx,by,bz;cx,cy,cz]
[[  5.947    0.      -3.4335]
 [ -0.       0.       6.867 ]
 [  0.     -19.807    0.    ]]

Structure factors:
|F(110)|^2 = 3.3963 barn
|F(100)|^2 = 0.0 barn
|F(1-10)|^2 = 0.0 barn
|F(001)|^2 = 0.0 barn
sigma_abs =129.00000000000003 barn/unit cell
sigma_inc =2.76 barn/unit cell


## Single Crystal SQW4 process. 

pyMACS includes an experimental UNION process, the Sqw4_process. It will work out of the box, and behaves very similarly to the single_crystal_process. It is essentially a port of Single_crystal_inelastic.comp by Duc Le. 

In [3]:
import os
import shutil
#To use the sqw4 process, it requires the copying of the component into the current directory. 
if not os.path.exists("Sqw4_process.comp"):
    shutil.copy(macs.instr_template_dir+"Sqw4_process.comp","Sqw4_process.comp")





In [4]:
#MACS can't actually access the full spinwave spectrum of this system, we will only 
# look at energy transfers below 12 meV. Ei_max = 17 meV

# In this example, the sample has three different scattering processes. 
# Bragg diffraction, incoherent scattering, and spin waves. 

scattering_def = ms.McStas_instr("scattering_definition",checks=False)
inc_scatter = scattering_def.add_component("inc_scatter","Incoherent_process")
inc_scatter.sigma=macs.sample.sigma_inc
inc_scatter.unit_cell_volume = macs.sample.cell_vol
inc_scatter.packing_factor = 1
inc_scatter.interact_fraction=1
inc_scatter.set_AT([0,0,0])

#Single crystal process. 
crystal_scatter = scattering_def.add_component("crystal_scatter","Single_crystal_process")
crystal_scatter.delta_d_d=0.005
crystal_scatter.mosaic = 30.0
#Projections of lattice vectors onto lab frame is handled by the previous helper process.
labproj = macs.sample.labframe_mat
crystal_scatter.ax = 6.867*np.sqrt(3)/2.0
crystal_scatter.ay = 0
crystal_scatter.az = -6.867*0.5
crystal_scatter.bx =0
crystal_scatter.by = 0
crystal_scatter.bz = 6.867
crystal_scatter.cx = 0
crystal_scatter.cy = 19.807
crystal_scatter.cz = 0
crystal_scatter.reflections='\"'+"CrI3.lau"+'\"'
crystal_scatter.barns=1
crystal_scatter.packing_factor=1
crystal_scatter.powder=0
crystal_scatter.PG=0
crystal_scatter.interact_fraction=1
crystal_scatter.set_AT([0,0,0])
crystal_scatter.set_ROTATED([0,0,0])

#Sample sqw4 processs
sample_sqw4 = scattering_def.add_component("sample_sqw4", "Sqw4_process")
sample_sqw4.sqw = '"spinw_CrI3_full.sqw4"'
#This file must also be added to the instrument file dependencies to be copied correctly.
macs.file_dependencies="spinw_CrI3_full.sqw4"
#We are choosing to have the (100) vector along the x-axis
sample_sqw4.ax=6.867*np.sqrt(3)/2.0
sample_sqw4.ay=0
sample_sqw4.az=-6.867*0.5
sample_sqw4.bx=0
sample_sqw4.by=0
sample_sqw4.bz=6.867
sample_sqw4.cx=0
sample_sqw4.cy=19.807
sample_sqw4.cz=0
sample_sqw4.aa=90
sample_sqw4.bb=90
sample_sqw4.cc=120
sample_sqw4.barns=1
sample_sqw4.max_stored_ki=1e5
sample_sqw4.max_bad=1e5
sample_sqw4.stored_dTheta = 0.1
sample_sqw4.stored_dkmag = 1e-4
sample_sqw4.recip_cell=0
sample_sqw4.interact_fraction=1
#sample_sqw4.init="'init'"
#sample_sqw4.append_EXTEND("// Remove direct beam\nif(!SCATTERED) ABSORB;")
sample_sqw4.set_AT([0,0,0])
sample_sqw4.set_ROTATED([0,0,0])

scattering = scattering_def.add_component("CrI3","Union_make_material")
scattering.process_string='"crystal_scatter,inc_scatter,sample_sqw4"'
scattering.my_absorption=macs.sample.rho_abs
scattering.set_AT([0,0,0])

#Now, this pseudo-instrument will be saved as the scattering definition of the sample. 
macs.sample.scattering_def = scattering_def

#Make a second object for the geometry. This particular case replicates the validation experiment for this package.
geo_def = ms.McStas_instr("geometry_definition",checks=False)

sample_cube=geo_def.add_component("sample_cube","Union_box")
sample_cube.xwidth=1.0*macs.sample.sample_widx
sample_cube.yheight=1.0*macs.sample.sample_widy
sample_cube.zdepth=1.0*macs.sample.sample_widz
sample_cube.priority=100
sample_cube.material_string='\"CrI3\"'
sample_cube.number_of_activations="number_of_activations_sample" #Do not change. 
sample_cube.set_AT([0,0,0],RELATIVE='crystal_assembly')
sample_cube.set_ROTATED([0,0,0],RELATIVE='crystal_assembly')

sample_plate = geo_def.add_component("sample_plate","Union_cylinder")
sample_plate.radius=0.006
sample_plate.yheight=0.002
sample_plate.priority=40
sample_plate.material_string='"Al"'
plate_distance = macs.sample.sample_widy+0.002
sample_plate.set_AT([0,plate_distance,0],RELATIVE="target")
sample_plate.set_ROTATED([0,0,0],RELATIVE="target")

sample_plate_rod = geo_def.add_component("sample_plate_rod","Union_cylinder")
sample_plate_rod.radius=0.00125
sample_plate_rod.yheight=0.0633
sample_plate_rod.priority=41
sample_plate_rod.material_string='"Al"'
sample_plate_rod.set_AT([0,plate_distance+0.001+0.031,0], RELATIVE="target")
sample_plate_rod.set_ROTATED([0,0,0],RELATIVE="target")

sample_base = geo_def.add_component("sample_base","Union_cylinder")
sample_base.radius=0.0065
sample_base.yheight=0.013
sample_base.priority=42
sample_base.material_string='\"Al\"'
sample_base.set_AT([0,0.0628,0],RELATIVE="target")
sample_base.set_ROTATED([0,0,0],RELATIVE="target")

sample_base_gap = geo_def.add_component("sample_base_gap","Union_cylinder")
sample_base_gap.radius=0.004
sample_base_gap.yheight=0.009
sample_base_gap.priority=43
sample_base_gap.material_string='"Vacuum"'
sample_base_gap.set_AT([0,0.0668,0], RELATIVE="target")
sample_base_gap.set_ROTATED([0,0,0],RELATIVE="target")

macs.sample.geometry_def = geo_def

In [7]:
#Some housekeeping steps needed for simulation
macs.prepare_expt_directory()
macs.edit_instr_file()
macs.compileMonochromator()
macs.compileInstr()

/mnt/c/Users/tjh/OneDrive - NIST/GitHub/pyMACS/docs/source/notebooks
#################

Starting compilation of monochromator.

Compilation of monochromator geometry successful.

#################

#################

Starting compilation of sample kidney geometry.

Compilation of sample kidney geometry successful.

#################



1

Perform the actual scan. This will take a long time.

In [9]:
#A note about file sizes here - a single 1e8 monochromator scan will make a 
# 350 Mb output file, so just be aware of this.
macs.n_mono=1e7
macs.n_sample=5e5


#The object is now set up and ready to perform simulations. 

macs.kidney.Ef=5.0
macs.kidney_angle_resolution=0.8
Ei_list = np.arange(5.0,17.0,1)


macs.preserve_kidney_scan_files=False
macs.script_scan(A3_list=np.arange(-120,120,0.8),Ei_list=Ei_list,\
                    num_threads=8,scan_title="cri3_a3scan_cube")

macs.data.load_data_matrix_from_csv("cri3_a3scan_cube_dataMatrix.csv")
macs.data.write_data_to_ng0(f"cri3_a3scan_cube.ng0")


Total Scans:   0%|          | 0/12 [00:00<?, ?it/s]

Ei=5.0 meV:   0%|          | 0/12 [00:00<?, ?it/s]

Ei=6.0 meV:   0%|          | 0/12 [00:00<?, ?it/s]

Ei=7.0 meV:   0%|          | 0/11 [00:00<?, ?it/s]

Ei=8.0 meV:   0%|          | 0/11 [00:00<?, ?it/s]

...

Ei=13.0 meV:   0%|          | 0/11 [00:00<?, ?it/s]

Ei=14.0 meV:   0%|          | 0/11 [00:00<?, ?it/s]

Ei=15.0 meV:   0%|          | 0/10 [00:00<?, ?it/s]

Ei=16.0 meV:   0%|          | 0/10 [00:00<?, ?it/s]

1